In [1]:
import pandas as pd

from sqlalchemy import create_engine

ck_properties_master = {  # clickhouse 数据库配置
    # "driver": "com.github.housepower.jdbc.ClickHouseDriver",
    "driver": "ru.yandex.clickhouse.ClickHouseDriver",
    "socket_timeout": "300000",
    "rewriteBatchedStatements": "true",
    "batchsize": "1000000",
    "numPartitions": "8",
    "user": "default",
    "password": "",
    "host": "10.129.165.72",
    "database": 'sdecdmp',
    "port": 8123
}

ck_engine_master = create_engine(
    f"clickhouse://{ck_properties_master['user']}:@{ck_properties_master['host']}:{ck_properties_master['port']}/{ck_properties_master['database']}"
)

In [2]:
history_sql = f"""
SELECT *
FROM
    sdecdmp.etl_data_macs
WHERE
    data_source = 'czdf' and clt_timestamp < 1684890587
"""
history_df = pd.read_sql(
    sql=history_sql,
    con=ck_engine_master
)
print(history_df.shape)
history_df.head()

(380259, 69)


,vin,ein,platname,data_source,data_type,clt_timestamp,clt_time,clt_date,hos_series,hos_mat_id,...,total_run_time,total_distance,total_fuel,PFltRgn_lSnceRgn,InjCrv_qPoI3Set,InjCtl_qCurr,Driving_regeneration,rgn_interval,Parking_regeneration,PFltRgn_stDem
0,L68211007P4004475,H9232006158,HD,czdf,gc,1684027906,2023-05-14 01:31:46,2023-05-14,None,None,...,0.15,0.0,1.0,0.0,-49.804688,73.950000,0,0.0,0,None
1,L68211007P4004475,H9232006158,HD,czdf,gc,1684027907,2023-05-14 01:31:47,2023-05-14,None,None,...,0.15,0.0,1.0,0.0,-49.804688,62.829998,0,0.0,0,None
2,L68211007P4004475,H9232006158,HD,czdf,gc,1684027908,2023-05-14 01:31:48,2023-05-14,None,None,...,0.15,0.0,1.0,0.0,-49.804688,26.240000,0,0.0,0,None
3,L68211007P4004475,H9232006158,HD,czdf,gc,1684027909,2023-05-14 01:31:49,2023-05-14,None,None,...,0.15,0.0,1.0,0.0,-49.804688,10.870000,0,0.0,0,None
4,L68211007P4004475,H9232006158,HD,czdf,gc,1684027910,2023-05-14 01:31:50,2023-05-14,None,None,...,0.15,0.0,1.0,0.0,-49.804688,10.230000,0,0.0,0,None


In [3]:
history_df.to_sql(
    name='etl_data_all',
    if_exists='append',
    index=False,
    con=ck_engine_master
)